In [ ]:
import cv2
import time
import random
import pygame
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()

pygame.init()

width, height = 800, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Mario")

mario_x = 100
mario_y = 660

jump_height = 20
gravity = 1
y_velocity = jump_height

goombas = []
goomba_timer = 0
goomba_interval = random.uniform(1.0,5.0)

goomba_x = 864
goomba_y = 660

goombas.append({"goomba_x":goomba_x,"goomba_y":goomba_y})

alternate = False

standing_surface = pygame.transform.scale(pygame.image.load("assets/mario_standing.png"), (48, 64))
jumping_surface = pygame.transform.scale(pygame.image.load("assets/mario_jumping.png"), (48, 64))
goomba_one = pygame.transform.scale(pygame.image.load("assets/goomba_one.png"), (64, 64))
goomba_two = pygame.transform.scale(pygame.image.load("assets/goomba_two.png"), (64, 64))
bg = pygame.image.load("assets/background.png")

mario_rectangle = standing_surface.get_rect(center=(mario_x,mario_y))
goomba_rectangle = goomba_one.get_rect(center=(goomba_x,goomba_y))

jumping = False
running = True

game_over = False
game_over_font = pygame.font.Font(None, 72)  
game_over_text = game_over_font.render("GAME OVER!", True, (0, 0, 0))  

score = 0
score_font = pygame.font.Font(None, 30)  
score_text = game_over_font.render(f"SCORE: {score}", True, (0, 0, 0))  

while running:
    state, frame = cap.read()
    
    if state:
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        screen.blit(bg,(0,0))

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                tip = [hand_landmarks.landmark[i].y for i in [8, 12, 16, 20]]
                dip = [hand_landmarks.landmark[i - 1].y for i in [8, 12, 16, 20]]

                opened = any(tip[i] < dip[i] for i in range(4))

                if opened and not jumping:
                    jumping = True
                    y_velocity = jump_height
                if not opened and jumping:
                    jumping = False

        if mario_y <= 660:
            if jumping:
                if mario_y <= 300:
                    mario_y = 300
                mario_y-=y_velocity
                y_velocity-=gravity
                if mario_y >= 660:
                    mario_y = 660
                    jumping = False
                    y_velocity = jump_height
                mario_rectangle = jumping_surface.get_rect(center=(mario_x,mario_y))
                screen.blit(jumping_surface,mario_rectangle)
            else:
                mario_y += jump_height
                if mario_y >= 660:
                    mario_y = 660
                mario_rectangle = standing_surface.get_rect(center=(mario_x,mario_y))
                screen.blit(standing_surface,mario_rectangle)

            current = time.time()
            if current - goomba_timer >= goomba_interval:
                goombas.append({"goomba_x":random.randint(864,1300),"goomba_y":660})
                goomba_timer = current
                goomba_interval = random.uniform(1.0,5.0)

            for goomba in goombas:
                goomba["goomba_x"] -= 10
                if goomba["goomba_x"] <= 0:
                    goombas.remove(goomba)
                if alternate:
                    goomba_rectangle = goomba_one.get_rect(center=(goomba["goomba_x"],goomba["goomba_y"]))
                    screen.blit(goomba_one,goomba_rectangle)
                    alternate = False
                else:
                    goomba_rectangle = goomba_two.get_rect(center=(goomba["goomba_x"],goomba["goomba_y"]))
                    screen.blit(goomba_two,goomba_rectangle)
                    alternate = True

                if mario_rectangle.colliderect(goomba_rectangle):
                    dx = mario_rectangle.centerx - goomba_rectangle.centerx
                    dy = mario_rectangle.centery - goomba_rectangle.centery

                    if abs(dx) > abs(dy):
                        if dx > 0:
                            game_over_rect = game_over_text.get_rect(center=(width // 2, height // 2))
                            screen.blit(game_over_text, game_over_rect)
                            game_over = True
                    else:
                        if dy > 0:
                            game_over = True
                        else:
                            score+=1
                            goombas.remove(goomba)
                            
        score_text = game_over_font.render(f"SCORE: {score}", True, (0, 0, 0))  
        score_rect = score_text.get_rect(center=(width // 2,50))
        screen.blit(score_text, score_rect)
        
        if game_over:
            goombas = []
            game_over_rect = game_over_text.get_rect(center=(width // 2, height // 2))
            screen.blit(game_over_text, game_over_rect)

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    cv2.imshow("JUMP", frame)

    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()
pygame.quit()
